# Reorganizing metadata for disease tracking

In [1]:
library(tidyverse)
library(vegan)
library(phyloseq)
library(dplyr)
library(ggplot2)
library(stringr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: permute

Loading required package: lattice

This is vegan 2.6-5



In [2]:
setwd('/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/metadata')

In [3]:
getwd()

[1] "/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/metadata"

In [4]:
colony_data <- read.csv('CBC_ColonyData.csv')

In [5]:
colony_data <- colony_data[, -1]

In [6]:
# extract year from Date_InitialTag and add it to new column
colony_data$Year_tagged <- format(as.Date(colony_data$Date_InitialTag, format="%m/%d/%y"), "%Y")

In [7]:
head(colony_data)

,Date_InitialTag,Transect,TransectNum,OldTagNum,NewTagNum,Species,Meter,Meters_90,Direction,Size_Class,⋯,Height,Date_DocumentedDisease,Date_DocumentedMortality,Notes_May2022,Sept_2023,Nov_2023,Dec_2023,Jan_2024,Feb_2024,Year_tagged
,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<int>,⋯,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,6/21/19,CBC30N,1,349,1,SSID,1.5,0.50,right,5,⋯,24,5/21/22,Diseased,NA,,Not Visited,Not Visited,,Not visited,2019
2,6/21/19,CBC30N,1,334,2,PAST,1.4,0.10,left,4,⋯,8,Healthy,Healthy,NA,90%Pale,100% Bleached,100% Bleached,Dead,"NOT dead- 90% mortality, too small for immune sample",2019
3,6/21/19,CBC30N,1,346,3,SSID,6.3,0.80,right,5,⋯,30,5/21/22,Diseased,NA,"80^Pale, 80% discoloration","95% Pale, 5% Bleached",80% CLP,,98% bleached,2019
4,10/15/19,CBC30N,1,407,4,PSTR,6.5,0.50,right,3,⋯,39,5/21/22,12/2/22,NA,,NA,NA,,Not visited,2019
5,6/24/19,CBC30N,1,394,5,SSID,4.3,1.20,left,5,⋯,27,5/21/22,Diseased,NA,,Not Visited,Not Visited,,Not visited,2019
6,10/15/19,CBC30N,1,413,6,PSTR,8.3,1.75,left,NA,⋯,38,12/2/22,9/25/23,NA,Dead,Not Visited,Not Visited,,Not visited,2019


In [8]:
nrow(colony_data)

[1] 186

## Changing Data Organization 

In [9]:
# Messing with reorganization to fully capture picture and timeline of disease and bleaching trajectory 
# Goal is to have 2 columns per date: MY_condition & MY_percentage (& probably date_notes col too)
# Using MonthYear format for date to keep consistent with sample labels 
# Percentage Notes: NA for healthy for now
# Condition Notes: put multiple conditions in the same colony if necessary, seperate by _

In [10]:
# getting subset so df is smaller to work with
#colony_conditions <- subset(colony_data, select = -c(Transect, OldTagNum, NewTagNum, Meter, Meters_90, Direction, MaxDiameter, Height))

# after manipulating data, need the entire df before downloading 
colony_conditions <- colony_data

In [11]:
head(colony_conditions)

,Date_InitialTag,Transect,TransectNum,OldTagNum,NewTagNum,Species,Meter,Meters_90,Direction,Size_Class,⋯,Height,Date_DocumentedDisease,Date_DocumentedMortality,Notes_May2022,Sept_2023,Nov_2023,Dec_2023,Jan_2024,Feb_2024,Year_tagged
,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<int>,⋯,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,6/21/19,CBC30N,1,349,1,SSID,1.5,0.50,right,5,⋯,24,5/21/22,Diseased,NA,,Not Visited,Not Visited,,Not visited,2019
2,6/21/19,CBC30N,1,334,2,PAST,1.4,0.10,left,4,⋯,8,Healthy,Healthy,NA,90%Pale,100% Bleached,100% Bleached,Dead,"NOT dead- 90% mortality, too small for immune sample",2019
3,6/21/19,CBC30N,1,346,3,SSID,6.3,0.80,right,5,⋯,30,5/21/22,Diseased,NA,"80^Pale, 80% discoloration","95% Pale, 5% Bleached",80% CLP,,98% bleached,2019
4,10/15/19,CBC30N,1,407,4,PSTR,6.5,0.50,right,3,⋯,39,5/21/22,12/2/22,NA,,NA,NA,,Not visited,2019
5,6/24/19,CBC30N,1,394,5,SSID,4.3,1.20,left,5,⋯,27,5/21/22,Diseased,NA,,Not Visited,Not Visited,,Not visited,2019
6,10/15/19,CBC30N,1,413,6,PSTR,8.3,1.75,left,NA,⋯,38,12/2/22,9/25/23,NA,Dead,Not Visited,Not Visited,,Not visited,2019


In [12]:
colnames(colony_data)

[1] "Date_InitialTag"          "Transect"                
 [3] "TransectNum"              "OldTagNum"               
 [5] "NewTagNum"                "Species"                 
 [7] "Meter"                    "Meters_90"               
 [9] "Direction"                "Size_Class"              
[11] "Notes_June_2019"          "MaxDiameter"             
[13] "Height"                   "Date_DocumentedDisease"  
[15] "Date_DocumentedMortality" "Notes_May2022"           
[17] "Sept_2023"                "Nov_2023"                
[19] "Dec_2023"                 "Jan_2024"                
[21] "Feb_2024"                 "Year_tagged"

In [13]:
# Add 2019_Coniditon and 2019_Percentage columns 
# If year tagged = 2019, "Healthy" in condition, NA in percentage
# If year tagged != 2019, NA for both
colony_conditions <- colony_conditions %>%
  mutate(`062019_Condition` = if_else(Year_tagged == 2019, "Healthy", NA_character_),
         `062019_Percentage` = NA_real_)

In [14]:
# Add 052022_Condition and 052022_Percentage columns 

# Look at DateDisease and DateMortality columns..if there's a may 2022 in either, add that condition, leave percentage blank for now
# If "Healthy" in datedisease, add healthy to condition 
# If year tagged != 2019 or 2022, NA for both

#establish pattern that selects colonies that got disease in May 2022
may_pattern <- grepl("5/\\d{1,2}/22", colony_conditions$Date_DocumentedDisease, fixed = FALSE)
may_dead_pattern <- grepl("5/\\d{1,2}/22", colony_conditions$Date_DocumentedMortality, fixed = FALSE)
tagged_may2022 <- grepl("5/\\d{1,2}/22", colony_conditions$Date_InitialTag, fixed = FALSE)
      
colony_conditions <- colony_conditions %>%
  mutate(`052022_Condition` =
           case_when(
             may_dead_pattern ~ "Dead",
             may_pattern ~ "Diseased",
             tagged_may2022 ~ "Healthy",
             TRUE ~ `062019_Condition`)  # Use 062019_Condition if none of the above conditions match
         )
#casewhen evaluates in order

In [15]:
# Add 122022_Condition and 122022_Percentage columns 

#establish pattern that selects colonies that got disease/died in Dec 2022
dec_pattern <- grepl("12/\\d{1,2}/22", colony_conditions$Date_DocumentedDisease, fixed = FALSE)
dec_dead_pattern <- grepl("12/\\d{1,2}/22", colony_conditions$Date_DocumentedMortality, fixed = FALSE)
tagged_dec2022 <- grepl("12/\\d{1,2}/22", colony_conditions$Date_InitialTag, fixed = FALSE)
      
colony_conditions <- colony_conditions %>%
  mutate(`122022_Condition` =
           case_when(
             dec_dead_pattern ~ "Dead",
               dec_pattern ~ "Diseased",
               tagged_dec2022 ~ "Healthy",
             TRUE ~ `052022_Condition`)  # Use 052022_Condition if none of the above conditions match
         )

In [16]:
# Add 2023_Condition and 2023_Percentage columns  

#establish pattern that selects colonies that got disease/died in 2023
may2023_pattern <- grepl("5/\\d{1,2}/23", colony_conditions$Date_DocumentedDisease, fixed = FALSE)
may2023_dead_pattern <- grepl("5/\\d{1,2}/23", colony_conditions$Date_DocumentedMortality, fixed = FALSE)

colony_conditions <- colony_conditions %>%
  mutate(
    `052023_Condition` = case_when(
      may2023_dead_pattern ~ "Dead",
      may2023_pattern ~ "Diseased",
      TRUE ~ `122022_Condition`
    )
  ) 

In [17]:
head(colony_conditions)

,Date_InitialTag,Transect,TransectNum,OldTagNum,NewTagNum,Species,Meter,Meters_90,Direction,Size_Class,⋯,Nov_2023,Dec_2023,Jan_2024,Feb_2024,Year_tagged,062019_Condition,062019_Percentage,052022_Condition,122022_Condition,052023_Condition
,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
1,6/21/19,CBC30N,1,349,1,SSID,1.5,0.50,right,5,⋯,Not Visited,Not Visited,,Not visited,2019,Healthy,NA,Diseased,Diseased,Diseased
2,6/21/19,CBC30N,1,334,2,PAST,1.4,0.10,left,4,⋯,100% Bleached,100% Bleached,Dead,"NOT dead- 90% mortality, too small for immune sample",2019,Healthy,NA,Healthy,Healthy,Healthy
3,6/21/19,CBC30N,1,346,3,SSID,6.3,0.80,right,5,⋯,"95% Pale, 5% Bleached",80% CLP,,98% bleached,2019,Healthy,NA,Diseased,Diseased,Diseased
4,10/15/19,CBC30N,1,407,4,PSTR,6.5,0.50,right,3,⋯,NA,NA,,Not visited,2019,Healthy,NA,Diseased,Dead,Dead
5,6/24/19,CBC30N,1,394,5,SSID,4.3,1.20,left,5,⋯,Not Visited,Not Visited,,Not visited,2019,Healthy,NA,Diseased,Diseased,Diseased
6,10/15/19,CBC30N,1,413,6,PSTR,8.3,1.75,left,NA,⋯,Not Visited,Not Visited,,Not visited,2019,Healthy,NA,Healthy,Diseased,Diseased


In [18]:
# sep 2023 is when bleaching tracking begins: take bleaching notes and incorporate into conditions 
colony_conditions_2023 <- colony_conditions %>%
rowwise() %>% 
  mutate(
    `092023_Condition` = case_when(
      str_detect(Sept_2023, "(?i)Pale") & str_detect(Sept_2023, "(?i)Bleach") & str_detect(Sept_2023, "(?i)Discolor") ~ "CLP,CLB,CLD",
        str_detect(Sept_2023, "(?i)Pale") & str_detect(Sept_2023, "(?i)Discolor") ~ "CLP,CLD",
        str_detect(Sept_2023, "(?i)Pale") & str_detect(Sept_2023, "(?i)Bleach") ~ "CLP,CLB",
        str_detect(Sept_2023, "(?i)Discolor")& str_detect(Sept_2023, "(?i)Bleach") ~ "CLB,CLD",
        str_detect(Sept_2023, "(?i)Pale") ~ "CLP",
        str_detect(Sept_2023, "(?i)Discolor") ~ "CLD",
        str_detect(Sept_2023, "(?i)Bleach") ~ "CLB",
        grepl("9/\\d{1,2}/23", Date_DocumentedDisease, fixed = FALSE) ~ "Diseased",
        grepl("9/\\d{1,2}/23", Date_DocumentedMortality, fixed = FALSE) ~ "Dead",
        TRUE ~ `052023_Condition`)
      )
        

rows<- colony_conditions_2023$Year_tagged == '2019'
head(colony_conditions_2023[rows,c("Date_DocumentedDisease","Date_DocumentedMortality","Sept_2023", "052023_Condition", "092023_Condition")])

Date_DocumentedDisease,Date_DocumentedMortality,Sept_2023,052023_Condition,092023_Condition
<chr>,<chr>,<chr>,<chr>,<chr>
5/21/22,Diseased,,Diseased,Diseased
Healthy,Healthy,90%Pale,Healthy,CLP
5/21/22,Diseased,"80^Pale, 80% discoloration",Diseased,"CLP,CLD"
5/21/22,12/2/22,,Dead,Dead
5/21/22,Diseased,,Diseased,Diseased
12/2/22,9/25/23,Dead,Diseased,Dead


In [19]:
colony_conditions_2023$Sept_2023 <-str_replace(colony_conditions_2023$Sept_2023, "\\^", "%")
colnames(colony_conditions_2023)

[1] "Date_InitialTag"          "Transect"                
 [3] "TransectNum"              "OldTagNum"               
 [5] "NewTagNum"                "Species"                 
 [7] "Meter"                    "Meters_90"               
 [9] "Direction"                "Size_Class"              
[11] "Notes_June_2019"          "MaxDiameter"             
[13] "Height"                   "Date_DocumentedDisease"  
[15] "Date_DocumentedMortality" "Notes_May2022"           
[17] "Sept_2023"                "Nov_2023"                
[19] "Dec_2023"                 "Jan_2024"                
[21] "Feb_2024"                 "Year_tagged"             
[23] "062019_Condition"         "062019_Percentage"       
[25] "052022_Condition"         "122022_Condition"        
[27] "052023_Condition"         "092023_Condition"

In [20]:
#make percentage column 092023_Percentage
colony_conditions_2023 <- colony_conditions_2023 %>%
  rowwise() %>% 
  mutate(
    `092023_Percentage` = ifelse(!is.na(Sept_2023), {
      percentages <- str_extract_all(Sept_2023, "\\b\\d+%")[[1]]
      reversed_percentages <- rev(percentages)
      paste(reversed_percentages, collapse = ",")
    }, "")
  )
rows<- colony_conditions_2023$Year_tagged == '2019'
colony_conditions_2023[rows,c("Date_DocumentedDisease","Date_DocumentedMortality","Sept_2023", "052023_Condition", "092023_Condition","092023_Percentage")]


Date_DocumentedDisease,Date_DocumentedMortality,Sept_2023,052023_Condition,092023_Condition,092023_Percentage
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
5/21/22,Diseased,,Diseased,Diseased,
Healthy,Healthy,90%Pale,Healthy,CLP,90%
5/21/22,Diseased,"80%Pale, 80% discoloration",Diseased,"CLP,CLD","80%,80%"
5/21/22,12/2/22,,Dead,Dead,
5/21/22,Diseased,,Diseased,Diseased,
12/2/22,9/25/23,Dead,Diseased,Dead,
Healthy,Healthy,,Healthy,Healthy,
5/21/22,9/25/23,Dead,Diseased,Dead,
5/21/22,Diseased,,Diseased,Diseased,


In [21]:
colony_conditions_2023[colony_conditions_2023$'092023_Percentage' != "",c("Sept_2023","092023_Condition","092023_Percentage")]

#test<-grepl("70",colony_conditions_2023$Sept_2023)
#colony_conditions_2023[test,c("Sept_2023","092023_Condition","092023_Percentage")]

Sept_2023,092023_Condition,092023_Percentage
<chr>,<chr>,<chr>
90%Pale,CLP,90%
"80%Pale, 80% discoloration","CLP,CLD","80%,80%"
10%Pale,CLP,10%
100%Pale,CLP,100%
100% Pale,CLP,100%
"100% Bleached, Sampled as Diseased",CLB,100%
"100% Bleached, Sampled as Diseased",CLB,100%
90%Pale,CLP,90%
"70%Bleach, 20% Pale","CLP,CLB","20%,70%"


In [22]:
colnames(colony_conditions_2023)


[1] "Date_InitialTag"          "Transect"                
 [3] "TransectNum"              "OldTagNum"               
 [5] "NewTagNum"                "Species"                 
 [7] "Meter"                    "Meters_90"               
 [9] "Direction"                "Size_Class"              
[11] "Notes_June_2019"          "MaxDiameter"             
[13] "Height"                   "Date_DocumentedDisease"  
[15] "Date_DocumentedMortality" "Notes_May2022"           
[17] "Sept_2023"                "Nov_2023"                
[19] "Dec_2023"                 "Jan_2024"                
[21] "Feb_2024"                 "Year_tagged"             
[23] "062019_Condition"         "062019_Percentage"       
[25] "052022_Condition"         "122022_Condition"        
[27] "052023_Condition"         "092023_Condition"        
[29] "092023_Percentage"

In [23]:
# make condition and percentage cols for Nov_2023
colony_conditions_2023 <- colony_conditions_2023 %>%
rowwise() %>% 
  mutate(
    `112023_Condition` = case_when(
      str_detect(Nov_2023, "(?i)Pale") & str_detect(Nov_2023, "(?i)Bleach") & str_detect(Nov_2023, "(?i)Discolor") ~ "CLP,CLD,CLB",
        str_detect(Nov_2023, "(?i)Pale") & str_detect(Nov_2023, "(?i)Discolor") ~ "CLP,CLD",
        str_detect(Nov_2023, "(?i)Pale") & str_detect(Nov_2023, "(?i)Bleach") ~ "CLP,CLB",
        str_detect(Nov_2023, "(?i)Discolor")& str_detect(Nov_2023, "(?i)Bleach") ~ "CLB,CLD",
        str_detect(Nov_2023, "(?i)Pale") ~ "CLP",
        str_detect(Nov_2023, "(?i)Discolor") ~ "CLD",
        str_detect(Nov_2023, "(?i)Bleach") ~ "CLB",
        grepl("11/\\d{1,2}/23", Date_DocumentedDisease, fixed = FALSE) ~ "Diseased",
        grepl("11/\\d{1,2}/23", Date_DocumentedMortality, fixed = FALSE) ~ "Dead",
        TRUE ~ `092023_Condition`)
      )

#make percentage column
colony_conditions_2023 <- colony_conditions_2023 %>%
  rowwise() %>% 
  mutate(
    `112023_Percentage` = ifelse(!is.na(Nov_2023), {
      percentages <- str_extract_all(Nov_2023, "\\b\\d+%")[[1]]
      reversed_percentages <- rev(percentages)
      paste(reversed_percentages, collapse = ",")
    }, "")
  )

head(colony_conditions_2023[, c("Date_DocumentedDisease", "Date_DocumentedMortality", "Sept_2023","Nov_2023", "052023_Condition", "092023_Condition", "112023_Condition","112023_Percentage")])
colony_conditions_2023[colony_conditions_2023$'112023_Percentage' != "",c("Nov_2023","112023_Condition","112023_Percentage")]


Date_DocumentedDisease,Date_DocumentedMortality,Sept_2023,Nov_2023,052023_Condition,092023_Condition,112023_Condition,112023_Percentage
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
5/21/22,Diseased,,Not Visited,Diseased,Diseased,Diseased,
Healthy,Healthy,90%Pale,100% Bleached,Healthy,CLP,CLB,100%
5/21/22,Diseased,"80%Pale, 80% discoloration","95% Pale, 5% Bleached",Diseased,"CLP,CLD","CLP,CLB","5%,95%"
5/21/22,12/2/22,,NA,Dead,Dead,Dead,
5/21/22,Diseased,,Not Visited,Diseased,Diseased,Diseased,
12/2/22,9/25/23,Dead,Not Visited,Diseased,Dead,Dead,


Nov_2023,112023_Condition,112023_Percentage
<chr>,<chr>,<chr>
100% Bleached,CLB,100%
"95% Pale, 5% Bleached","CLP,CLB","5%,95%"
100% Pale,CLP,100%
98% Mortality,Healthy,98%
100% Pale,CLP,100%
100% Bleached,CLB,100%
5% Bleached,CLB,5%
90% Bleached,CLB,90%
100% Pale,CLP,100%


In [24]:
cols_to_look <- c('Notes_June_2019', 'Notes_May2022', 'Sept_2023', 'Nov_2023', 'Dec_2023', 'Jan_2024', 'Feb_2024')
#colony_conditions_2023[,cols_to_look]
rows_mortality<-str_detect(colony_conditions_2023[,cols_to_look], "Mortality|TL")
#colony_conditions_2023[rows_mortality,]

Warning message in stri_detect_regex(string, pattern, negate = negate, opts_regex = opts(pattern)):
“argument is not an atomic vector; coercing”


In [25]:
find <- str_detect(colony_conditions_2023$Nov_2023, "Mortality|TL")
colony_conditions_2023[find, "Nov_2023"]

Nov_2023
<chr>
NA
NA
NA
98% Mortality
NA
NA
NA
NA
NA


In [26]:
# make condition and percentage cols for Nov_2023
colony_conditions_2023 <- colony_conditions_2023 %>%
rowwise() %>% 
  mutate(
    `122023_Condition` = case_when(
      str_detect(Dec_2023, "(?i)Pale|CLP") & str_detect(Dec_2023, "(?i)Bleach|CLB") & str_detect(Dec_2023, "(?i)Discolor|CLD") ~ "CLP,CLD,CLB",
        str_detect(Dec_2023, "(?i)Pale|CLP") & str_detect(Dec_2023, "(?i)Discolor|CLD") ~ "CLP,CLD",
        str_detect(Dec_2023, "(?i)Pale|CLP|Paling") & str_detect(Dec_2023, "(?i)Bleach|CLB") ~ "CLP,CLB",
        str_detect(Dec_2023, "(?i)Discolor|CLD")& str_detect(Dec_2023, "(?i)Bleach|CLB") ~ "CLB,CLD",
        str_detect(Dec_2023, "(?i)Pale|CLP|Paling") ~ "CLP",
        str_detect(Dec_2023, "(?i)Discolor|CLD") ~ "CLD",
        str_detect(Dec_2023, "(?i)Bleach|CLB") ~ "CLB",
        grepl("12/\\d{1,2}/23", Date_DocumentedDisease, fixed = FALSE) ~ "Diseased",
        grepl("12/\\d{1,2}/23", Date_DocumentedMortality, fixed = FALSE) ~ "Dead",
        TRUE ~ `112023_Condition`)
      )

#make percentage column
colony_conditions_2023 <- colony_conditions_2023 %>%
  rowwise() %>% 
  mutate(
    `122023_Percentage` = ifelse(!is.na(Dec_2023), {
      percentages <- str_extract_all(Dec_2023, "\\b\\d+%")[[1]]
      reversed_percentages <- rev(percentages)
      paste(reversed_percentages, collapse = ",")
    }, "")
  )

head(colony_conditions_2023[, c("Date_DocumentedDisease", "Date_DocumentedMortality","Nov_2023", "Dec_2023", "112023_Condition","112023_Percentage","122023_Condition","122023_Percentage" )])
Dec2023_conditions<- colony_conditions_2023[colony_conditions_2023$'122023_Percentage' != "",c("Dec_2023","122023_Condition","122023_Percentage")]


Date_DocumentedDisease,Date_DocumentedMortality,Nov_2023,Dec_2023,112023_Condition,112023_Percentage,122023_Condition,122023_Percentage
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
5/21/22,Diseased,Not Visited,Not Visited,Diseased,,Diseased,
Healthy,Healthy,100% Bleached,100% Bleached,CLB,100%,CLB,100%
5/21/22,Diseased,"95% Pale, 5% Bleached",80% CLP,"CLP,CLB","5%,95%",CLP,80%
5/21/22,12/2/22,NA,NA,Dead,,Dead,
5/21/22,Diseased,Not Visited,Not Visited,Diseased,,Diseased,
12/2/22,9/25/23,Not Visited,Not Visited,Dead,,Dead,


In [27]:
rows_to_look_at<-str_detect(Dec2023_conditions$Dec_2023, "Mortality|TL")
Dec2023_conditions$`122023_Percentage`[rows_to_look_at] <- c("20%", "100%", "100%")
Dec2023_conditions[rows_to_look_at,]

Dec_2023,122023_Condition,122023_Percentage
<chr>,<chr>,<chr>
"20% CLP, 10% TL",CLP,20%
"90% Mortality, 100% CLB",CLB,100%
"75% Mortality, 100%CLB",CLB,100%


In [28]:
# make condition and percentage cols for Nov_2023
colony_conditions_2023 <- colony_conditions_2023 %>%
rowwise() %>% 
  mutate(
    `012024_Condition` = case_when(
      str_detect(Jan_2024, "(?i)Pale|CLP") & str_detect(Jan_2024, "(?i)Bleach|CLB") & str_detect(Jan_2024, "(?i)Discolor|CLD") ~ "CLP,CLD,CLB",
        str_detect(Jan_2024, "(?i)Pale|CLP") & str_detect(Jan_2024, "(?i)Discolor|CLD") ~ "CLP,CLD",
        str_detect(Jan_2024, "(?i)Pale|CLP|Paling") & str_detect(Jan_2024, "(?i)Bleach|CLB") ~ "CLP,CLB",
        str_detect(Jan_2024, "(?i)Discolor|CLD")& str_detect(Jan_2024, "(?i)Bleach|CLB") ~ "CLB,CLD",
        str_detect(Jan_2024, "(?i)Pale|CLP|Paling") ~ "CLP",
        str_detect(Jan_2024, "(?i)Discolor|CLD") ~ "CLD",
        str_detect(Jan_2024, "(?i)Bleach|CLB") ~ "CLB",
        grepl("1/\\d{1,2}/24", Date_DocumentedDisease, fixed = FALSE) ~ "Diseased",
        grepl("1/\\d{1,2}/24", Date_DocumentedMortality, fixed = FALSE) ~ "Dead",
        TRUE ~ `122023_Condition`)
      )

#make percentage column
colony_conditions_2023 <- colony_conditions_2023 %>%
  rowwise() %>% 
  mutate(
    `012024_Percentage` = ifelse(!is.na(Jan_2024), {
      percentages <- str_extract_all(Jan_2024, "\\b\\d+%")[[1]]
      reversed_percentages <- rev(percentages)
      paste(reversed_percentages, collapse = ",")
    }, "")
  )

head(colony_conditions_2023[, c("Date_DocumentedDisease", "Date_DocumentedMortality","Dec_2023", "Jan_2024","122023_Condition","122023_Percentage", "012024_Condition", "012024_Percentage" )])
colony_conditions_2023[colony_conditions_2023$'012024_Percentage' != "",c("Jan_2024","012024_Condition","012024_Percentage")]


Date_DocumentedDisease,Date_DocumentedMortality,Dec_2023,Jan_2024,122023_Condition,122023_Percentage,012024_Condition,012024_Percentage
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
5/21/22,Diseased,Not Visited,,Diseased,,Diseased,
Healthy,Healthy,100% Bleached,Dead,CLB,100%,CLB,
5/21/22,Diseased,80% CLP,,CLP,80%,CLP,
5/21/22,12/2/22,NA,,Dead,,Dead,
5/21/22,Diseased,Not Visited,,Diseased,,Diseased,
12/2/22,9/25/23,Not Visited,,Dead,,Dead,


Jan_2024,012024_Condition,012024_Percentage
<chr>,<chr>,<chr>
60% Bleached,CLB,60%
85% Bleached,CLB,85%
20% Bleached,CLB,20%
10%TL,CLP,10%
5% CLP,CLP,5%
10% Bleaching,CLB,10%
5% CLP,CLP,5%
"5% CLP, damsel predation",CLP,5%
1% CLP,CLP,1%


In [29]:
# make condition and percentage cols for Nov_2023
colony_conditions_2023 <- colony_conditions_2023 %>%
rowwise() %>% 
  mutate(
    `022024_Condition` = case_when(
      str_detect(Feb_2024, "(?i)Pale|CLP") & str_detect(Feb_2024, "(?i)Bleach|CLB") & str_detect(Feb_2024, "(?i)Discolor|CLD") ~ "CLP,CLD,CLB",
        str_detect(Feb_2024, "(?i)Pale|CLP") & str_detect(Feb_2024, "(?i)Discolor|CLD") ~ "CLP,CLD",
        str_detect(Feb_2024, "(?i)Pale|CLP|Paling") & str_detect(Feb_2024, "(?i)Bleach|CLB") ~ "CLP,CLB",
        str_detect(Feb_2024, "(?i)Discolor|CLD")& str_detect(Feb_2024, "(?i)Bleach|CLB") ~ "CLB,CLD",
        str_detect(Feb_2024, "(?i)Pale|CLP|Paling") ~ "CLP",
        str_detect(Feb_2024, "(?i)Discolor|CLD") ~ "CLD",
        str_detect(Feb_2024, "(?i)Bleach|CLB") ~ "CLB",
        grepl("2/\\d{1,2}/24", Date_DocumentedDisease, fixed = FALSE) ~ "Diseased",
        grepl("2/\\d{1,2}/24", Date_DocumentedMortality, fixed = FALSE) ~ "Dead",
        TRUE ~ `012024_Condition`)
      )

#make percentage column
colony_conditions_2023 <- colony_conditions_2023 %>%
  rowwise() %>% 
  mutate(
    `022024_Percentage` = ifelse(!is.na(Feb_2024), {
      percentages <- str_extract_all(Feb_2024, "\\b\\d+%")[[1]]
      reversed_percentages <- rev(percentages)
      paste(reversed_percentages, collapse = ",")
    }, "")
  )

head(colony_conditions_2023[, c("Date_DocumentedDisease", "Date_DocumentedMortality", "Jan_2024","Feb_2024", "012024_Condition", "012024_Condition","022024_Condition", "022024_Percentage" )])
colony_conditions_2023[colony_conditions_2023$'022024_Percentage' != "",c("Feb_2024","022024_Condition","022024_Percentage")]


Date_DocumentedDisease,Date_DocumentedMortality,Jan_2024,Feb_2024,012024_Condition,012024_Condition,022024_Condition,022024_Percentage
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
5/21/22,Diseased,,Not visited,Diseased,Diseased,Diseased,
Healthy,Healthy,Dead,"NOT dead- 90% mortality, too small for immune sample",CLB,CLB,CLB,90%
5/21/22,Diseased,,98% bleached,CLP,CLP,CLB,98%
5/21/22,12/2/22,,Not visited,Dead,Dead,Dead,
5/21/22,Diseased,,Not visited,Diseased,Diseased,Diseased,
12/2/22,9/25/23,,Not visited,Dead,Dead,Dead,


Feb_2024,022024_Condition,022024_Percentage
<chr>,<chr>,<chr>
"NOT dead- 90% mortality, too small for immune sample",CLB,90%
98% bleached,CLB,98%
"95% mortality, too small for immune sample",CLP,95%
"25% pale, some mortality, not in great shape",CLP,25%
"Diseased, 30% bleached, 10% pale","CLP,CLB","10%,30%"
5% bleached,CLB,5%
"5% pale, some small old mortality patches on the top?",CLP,5%
"10% pale, some damselfish predation",CLP,10%


In [30]:
getwd()

[1] "/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/metadata"

In [31]:
head(colony_conditions_2023)

Date_InitialTag,Transect,TransectNum,OldTagNum,NewTagNum,Species,Meter,Meters_90,Direction,Size_Class,⋯,092023_Condition,092023_Percentage,112023_Condition,112023_Percentage,122023_Condition,122023_Percentage,012024_Condition,012024_Percentage,022024_Condition,022024_Percentage
<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
6/21/19,CBC30N,1,349,1,SSID,1.5,0.50,right,5,⋯,Diseased,,Diseased,,Diseased,,Diseased,,Diseased,
6/21/19,CBC30N,1,334,2,PAST,1.4,0.10,left,4,⋯,CLP,90%,CLB,100%,CLB,100%,CLB,,CLB,90%
6/21/19,CBC30N,1,346,3,SSID,6.3,0.80,right,5,⋯,"CLP,CLD","80%,80%","CLP,CLB","5%,95%",CLP,80%,CLP,,CLB,98%
10/15/19,CBC30N,1,407,4,PSTR,6.5,0.50,right,3,⋯,Dead,,Dead,,Dead,,Dead,,Dead,
6/24/19,CBC30N,1,394,5,SSID,4.3,1.20,left,5,⋯,Diseased,,Diseased,,Diseased,,Diseased,,Diseased,
10/15/19,CBC30N,1,413,6,PSTR,8.3,1.75,left,NA,⋯,Dead,,Dead,,Dead,,Dead,,Dead,


In [32]:
colnames(colony_conditions_2023)

[1] "Date_InitialTag"          "Transect"                
 [3] "TransectNum"              "OldTagNum"               
 [5] "NewTagNum"                "Species"                 
 [7] "Meter"                    "Meters_90"               
 [9] "Direction"                "Size_Class"              
[11] "Notes_June_2019"          "MaxDiameter"             
[13] "Height"                   "Date_DocumentedDisease"  
[15] "Date_DocumentedMortality" "Notes_May2022"           
[17] "Sept_2023"                "Nov_2023"                
[19] "Dec_2023"                 "Jan_2024"                
[21] "Feb_2024"                 "Year_tagged"             
[23] "062019_Condition"         "062019_Percentage"       
[25] "052022_Condition"         "122022_Condition"        
[27] "052023_Condition"         "092023_Condition"        
[29] "092023_Percentage"        "112023_Condition"        
[31] "112023_Percentage"        "122023_Condition"        
[33] "122023_Percentage"        "012024_Condition"        
[35] "012024_Percentage"        "022024_Condition"        
[37] "022024_Percentage"

In [33]:
colnames(colony_data)

[1] "Date_InitialTag"          "Transect"                
 [3] "TransectNum"              "OldTagNum"               
 [5] "NewTagNum"                "Species"                 
 [7] "Meter"                    "Meters_90"               
 [9] "Direction"                "Size_Class"              
[11] "Notes_June_2019"          "MaxDiameter"             
[13] "Height"                   "Date_DocumentedDisease"  
[15] "Date_DocumentedMortality" "Notes_May2022"           
[17] "Sept_2023"                "Nov_2023"                
[19] "Dec_2023"                 "Jan_2024"                
[21] "Feb_2024"                 "Year_tagged"

In [34]:
write.csv(colony_conditions_2023, '/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/metadata/CBC_ColonyData_replacement.csv', row.names = FALSE)


In [35]:
#subset(colony_conditions_2023, is.na(`Date_DocumentedDisease`))
#Deleting rows that have na for both datedisease and datemortality

In [30]:
#colony_conditions_2023<- subset(colony_conditions_2023, !(is.na(`Date_DocumentedDisease`) & is.na(`Date_DocumentedMortality`)))